In [1]:
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

['data/names\\Arabic.txt', 'data/names\\Chinese.txt', 'data/names\\Czech.txt', 'data/names\\Dutch.txt', 'data/names\\English.txt', 'data/names\\French.txt', 'data/names\\German.txt', 'data/names\\Greek.txt', 'data/names\\Indian.txt', 'data/names\\Irish.txt', 'data/names\\Italian.txt', 'data/names\\Japanese.txt', 'data/names\\Korean.txt', 'data/names\\Polish.txt', 'data/names\\Portuguese.txt', 'data/names\\Russian.txt', 'data/names\\Scottish.txt', 'data/names\\Spanish.txt', 'data/names\\Vietnamese.txt']


In [2]:
import unicodedata
import string

all_letters = string.ascii_letters + ".,;"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

Slusarski


In [3]:
category_lines = {}
all_categories = []

def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

print(category_lines['Indian'][:5])

['Aarav', 'Vivaan', 'Aditya', 'Vihaan', 'Arjun']


In [4]:
import torch

def letterToIndex(letter):
    return all_letters.find(letter)

def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Vivan').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]])
torch.Size([5, 1, 55])


In [5]:
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        hidden = F.tanh(self.i2h(input) + self.h2h(hidden))
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
    
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

print(rnn)

RNN(
  (i2h): Linear(in_features=55, out_features=128, bias=True)
  (h2h): Linear(in_features=128, out_features=128, bias=True)
  (h2o): Linear(in_features=128, out_features=19, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [7]:
input = lineToTensor('Aarav')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

tensor([[-3.0176, -3.0131, -2.9598, -3.0492, -3.0048, -3.0316, -2.9825, -2.9209,
         -2.8656, -2.8507, -2.9849, -2.9003, -2.8588, -2.8056, -3.0165, -2.9738,
         -2.8947, -2.9535, -2.9056]], grad_fn=<LogSoftmaxBackward0>)
('Polish', 13)


In [8]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Arabic / line = Sleiman
category = Greek / line = Sakellariou
category = Russian / line = Matzeyovsky
category = Vietnamese / line = Pho
category = Scottish / line = Millar
category = Portuguese / line = Freitas
category = Russian / line = Homutnikov
category = Polish / line = Rutkowski
category = Indian / line = Aarohi
category = German / line = Buchholz


In [9]:
criterion = nn.NLLLoss()

learning_rate = 0.001

In [10]:
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [11]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000


current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

In [12]:
for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 10s) 2.9451 Horoshevsky / Greek ✗ (Russian)
10000 10% (0m 21s) 2.7786 Kacirek / Czech ✓
15000 15% (0m 39s) 3.1462 Churchill / Polish ✗ (English)
20000 20% (0m 52s) 2.0282 Ryoo / Korean ✓
25000 25% (1m 11s) 1.5241 Zhui / Chinese ✓
30000 30% (1m 55s) 2.8014 Asfour / German ✗ (Arabic)
35000 35% (2m 32s) 2.6789 Dawes / Portuguese ✗ (English)
40000 40% (2m 43s) 0.9126 Kinugasa / Japanese ✓
45000 45% (2m 53s) 2.3788 Vaishnavi / Japanese ✗ (Indian)
50000 50% (3m 21s) 2.4957 Firmin / Irish ✗ (French)
55000 55% (3m 53s) 1.7007 Salib / Arabic ✓
60000 60% (4m 13s) 0.9847 She / Chinese ✓
65000 65% (4m 31s) 3.0173 Close / French ✗ (Greek)
70000 70% (4m 44s) 1.5122 Mcdonald / Scottish ✓
75000 75% (4m 57s) 1.2752 Doan / Vietnamese ✓
80000 80% (5m 10s) 0.8752 Ping / Chinese ✓
85000 85% (5m 23s) 1.5817 Cardona / Italian ✗ (Spanish)
90000 90% (5m 35s) 1.0609 Faucheux / French ✓
95000 95% (5m 48s) 0.6675 Luong / Vietnamese ✓
100000 100% (6m 1s) 2.5503 Tanmay / Arabic ✗ (Indian)


In [13]:
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

In [18]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('Arjun')


> Arjun
(-1.32) Indian
(-1.65) Arabic
(-1.89) Irish
